In [1]:
year = 2024; day = 24

In [2]:
from aocd import get_data, submit
data = get_data(year=year, day=day)

data = data.strip()
initial_state, circuit = data.split("\n\n")

state = {}
for line in initial_state.split("\n"):
    line = line.split(":")
    state[line[0]] = int(line[1])

circ = []
for line in circuit.split("\n"):
    line = line.split()
    circ.append((line[0], line[2], line[1], line[4]))

In [3]:
def parse_circuit(state, circ):
    parsed = set()
    for _ in range(100):
        for instr in circ:
            if instr in parsed:
                continue
            a, b, op, res = instr
            if not (a in state and b in state):
                continue
            parsed.add(instr)
            if op == "AND":
                state[res] = state[a] & state[b]
            elif op == "OR":
                state[res] = state[a] | state[b]
            elif op == "XOR":
                state[res] = state[a] ^ state[b]
        if len(parsed) == len(circ):
            break
    else:
        raise ValueError("max iterations exceeded")
    return state

In [5]:
s = parse_circuit(state, circ)

output_size = max(int(k[1:])+1 for k in s.keys() if k.startswith('z'))
output = [s[f"z{i:02d}"] for i in range(output_size)]
out_str = "".join(map(str, reversed(output)))
ans1 = int(out_str, 2)

In [6]:
submit(ans1, part="a", year=year, day=day)

Part a already solved with same answer: 42883464055378


In [7]:
# BINARY ADDER
# interim_bit = a XOR b -- where a,b should startwith [x,y]
# and_bit = a AND b -- where a,b should startwith [x,y]
# sum_bit = interim_bit XOR carry
# interim_carry = carry_in AND interim_bit
# carry = and_bit OR interim_carry

#inputs: 45
#outputs: 46
#carries: 44

def is_input(a):
    return a.startswith('x') or a.startswith('y')

In [8]:
interim_bits = set()
output_bits = set()
and_bits = set()
interim_carry_bits = set()
carry_bits = set()
for instr in circ:
    a, b, op, res = instr
    if op == "XOR":
        if is_input(a) and is_input(b) and not res == "z00":
            #input bits should not have been swapped
            assert a[1:] == b[1:]
            interim_bits.add(res)
        else:
            output_bits.add(res)
    if op == "AND":
        if is_input(a) and is_input(b):
            assert a[1:] == b[1:]
            if a[1:] == "00":
                carry_bits.add(res)
            else:
                and_bits.add(res)
        else:
            interim_carry_bits.add(res)
    if op == "OR":
        carry_bits.add(res)

print(
    f"interim_bits: {len(interim_bits)}",
    f"output_bits: {len(output_bits)}",
    f"and_bits: {len(and_bits)}",
    f"interim_carry_bits: {len(interim_carry_bits)}",
    f"carry_bits: {len(carry_bits)}",
)

interim_bits: 44 output_bits: 45 and_bits: 44 interim_carry_bits: 44 carry_bits: 45


In [9]:
to_swap = set()

In [10]:
to_swap.update([a for a in output_bits if not a.startswith('z')])
to_swap.update([a for a in interim_bits if a.startswith('z')])
to_swap.update([a for a in interim_carry_bits if a.startswith('z')])
to_swap.update([a for a in and_bits if a.startswith('z')])
to_swap.update([a for a in carry_bits if a.startswith('z') and a != 'z45'])

In [11]:
for instr in circ:
    a, b, op, res = instr
    if res in output_bits:
        if res == "z00":
            continue
        if not (a in interim_bits or b in interim_bits) or not (a in carry_bits or b in carry_bits):
            if a in carry_bits or a in interim_bits:
                to_swap.add(b)
            else:
                to_swap.add(a)
for instr in circ:
    a, b, op, res = instr
    if res in carry_bits:
        if not (a in interim_carry_bits or b in interim_carry_bits) or not (a in and_bits or b in and_bits):
            if a == "x00" or b == "x00":
                continue
            if a in and_bits or a in interim_carry_bits:
                to_swap.add(b)
            else:
                to_swap.add(a)


In [12]:
ans2 = ",".join(sorted(list(to_swap)))

In [13]:
submit(ans2, part="b", day=day, year=year)

Part b already solved with same answer: dqr,dtk,pfw,shh,vgs,z21,z33,z39


In [ ]:
to_number = lambda bits:int("".join(map(str, reversed(bits))), 2)

input_x = to_number([s[f"x{i:02d}"] for i in range(45)])
input_y = to_number([s[f"y{i:02d}"] for i in range(45)])
output = to_number([s[f"z{i:02d}"] for i in range(46)])

expected = input_x + input_y

In [35]:
def update_circ(pairs, circ):
    new_circ = []
    for instr in circ:
        instr = ",".join(instr)
        for a1, a2 in pairs:
            instr = instr.replace(a1, "TMP")
            instr = instr.replace(a2, a1)
            instr = instr.replace("TMP", a2)
        new_circ.append(tuple(instr.split(",")))
    return new_circ


In [45]:
from itertools import combinations, permutations
count = 0
found = False
for a1, b1, c1, d1 in combinations(to_swap, 4):
    others = to_swap - {a1, b1, c1, d1}
    for a2, b2, c2, d2 in permutations(others, 4):
        count += 1
        pairs = ((a1, a2), (b1, b2), (c1, c2), (d1, d2))
        c = update_circ(pairs, circ)
        s = parse_circuit(state, c)
        output = to_number([s[f"z{i:02d}"] for i in range(46)])
        if output == expected:
            print(pairs)
            found = True
        if found:
            break
    if found: break
count

(('z33', 'dqr'), ('vgs', 'z21'), ('dtk', 'pfw'), ('z39', 'shh'))


865